In [1]:
import pandas as pd
import numpy as np


ModuleNotFoundError: No module named 'pandas'

In [2]:
df_train = pd.read_csv("../data/news/train.csv")
df_test = pd.read_csv("../data/news/test.csv")

df_train.replace(np.nan, "", inplace=True)
df_test.replace(np.nan, "", inplace=True)

df_train.loc[:,'text'] = df_train.apply(lambda row: row['title']+" "+ row['desc'], axis=1)
df_test.loc[:,'text'] = df_test.apply(lambda row: row['title']+" "+ row['desc'], axis=1)

train_txt = " ".join (df_train.text)
test_txt = " ".join (df_test.text)

SAVE_DIR = "../data/news"

file1 = open(f"{SAVE_DIR}/train.txt","w")
file1.write(train_txt) 
file1.close()

file2 = open(f"{SAVE_DIR}/test.txt","w")
file2.write(test_txt) 
file2.close()

In [3]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [7]:
from transformers import BertForMaskedLM

model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
model.num_parameters()

109514298

In [10]:
%%time
#building the training dataset
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="../data/news/train.csv",
    block_size=128,
)

CPU times: user 4.84 s, sys: 222 ms, total: 5.06 s
Wall time: 676 ms


In [11]:
#data collector to help us batch differetn samples of dataset together
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [23]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="../model_weights/news_BERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [24]:
%%time
trainer.train()

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/users/grad/frahimi/.local/lib/python3.7/site-packages/torch/nn/parallel/parallel_apply.py", line 60, in _worker
    output = module(*input, **kwargs)
  File "/users/grad/frahimi/.local/lib/python3.7/site-packages/torch/nn/modules/module.py", line 550, in __call__
    result = self.forward(*input, **kwargs)
  File "/users/grad/frahimi/.local/lib/python3.7/site-packages/transformers/models/bert/modeling_bert.py", line 1307, in forward
    return_dict=return_dict,
  File "/users/grad/frahimi/.local/lib/python3.7/site-packages/torch/nn/modules/module.py", line 550, in __call__
    result = self.forward(*input, **kwargs)
  File "/users/grad/frahimi/.local/lib/python3.7/site-packages/transformers/models/bert/modeling_bert.py", line 968, in forward
    return_dict=return_dict,
  File "/users/grad/frahimi/.local/lib/python3.7/site-packages/torch/nn/modules/module.py", line 550, in _